In [16]:
# model
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

import numpy as np
import os

# dataset and transformation
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import models
import os

# display images
from torchvision import utils
import matplotlib.pyplot as plt
%matplotlib inline

# utils
import numpy as np
import time
import copy

In [17]:
data_train = []
data_test = []

def classTrainData(): # set data_train
    path_dir = './res_data/train'
    file_list = os.listdir(path_dir)
    for data in file_list:
      temp = np.load('./res_data/train/' + data)
      tempData = torch.Tensor(temp)
      if "AD" in data:
        temp = (tempData, 0)
        data_train.append(temp)
      elif "NM" in data:
        temp = (tempData, 1)
        data_train.append(temp)
      elif "PD" in data:
        temp = (tempData, 2)
        data_train.append(temp)

def classTestData(): # set data_train
    path_dir = './res_data/test'
    file_list = os.listdir(path_dir)
    for data in file_list:
      temp = np.load('./res_data/test/' + data)
      tempData = torch.Tensor(temp)
      if "AD" in data:
        temp = (tempData, 0)
        data_test.append(temp)
      elif "NM" in data:
        temp = (tempData, 1)
        data_test.append(temp)
      elif "PD" in data:
        temp = (tempData, 2)
        data_test.append(temp)

In [18]:
classTrainData()
classTestData()

In [5]:
# define transformation
transformation = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Resize(224)
])

# apply transformation to dataset
for i in range(0, len(data_train)):
    data_train[i][0].transform = transformation
for i in range(0, len(data_test):
    data_test[i][0].transform = transformation

# make dataloader
train_dl = DataLoader(data_train, batch_size=10, shuffle=True)
val_dl = DataLoader(data_test, batch_size=10, shuffle=True)

In [6]:
# Swish activation function
class Swish(nn.Module):
    def __init__(self):
        super().__init__()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return x * self.sigmoid(x)

# check
if __name__ == '__main__':
    x = torch.randn(4, 4, 224, 224)
    model = Swish()
    output = model(x)
    print('output size:', output.size())
    
    # SE Block
class SEBlock(nn.Module):
    def __init__(self, in_channels, r=4):
        super().__init__()

        self.squeeze = nn.AdaptiveAvgPool2d((1,1))
        self.excitation = nn.Sequential(
            nn.Linear(in_channels, in_channels * r),
            Swish(),
            nn.Linear(in_channels * r, in_channels),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.squeeze(x)
        x = x.view(x.size(0), -1)
        x = self.excitation(x)
        x = x.view(x.size(0), x.size(1), 1, 1)
        return x

# check
if __name__ == '__main__':
    x = torch.randn(4, 56, 17, 17)
    model = SEBlock(x.size(1))
    output = model(x)
    print('output size:', output.size())
    
class MBConv(nn.Module):
    expand = 6
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, se_scale=4, p=0.5):
        super().__init__()
        # first MBConv is not using stochastic depth
        self.p = torch.tensor(p).float() if (in_channels == out_channels) else torch.tensor(1).float()

        self.residual = nn.Sequential(
            nn.Conv2d(in_channels, in_channels * MBConv.expand, 1, stride=stride, padding=0, bias=False),
            nn.BatchNorm2d(in_channels * MBConv.expand, momentum=0.99, eps=1e-3),
            Swish(),
            nn.Conv2d(in_channels * MBConv.expand, in_channels * MBConv.expand, kernel_size=kernel_size,
                      stride=1, padding=kernel_size//2, bias=False, groups=in_channels*MBConv.expand),
            nn.BatchNorm2d(in_channels * MBConv.expand, momentum=0.99, eps=1e-3),
            Swish()
        )

        self.se = SEBlock(in_channels * MBConv.expand, se_scale)

        self.project = nn.Sequential(
            nn.Conv2d(in_channels*MBConv.expand, out_channels, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(out_channels, momentum=0.99, eps=1e-3)
        )

        self.shortcut = (stride == 1) and (in_channels == out_channels)

    def forward(self, x):
        # stochastic depth
        if self.training:
            if not torch.bernoulli(self.p):
                return x

        x_shortcut = x
        x_residual = self.residual(x)
        x_se = self.se(x_residual)

        x = x_se * x_residual
        x = self.project(x)

        if self.shortcut:
            x= x_shortcut + x

        return x

# check
if __name__ == '__main__':
    x = torch.randn(4, 16, 24, 24)
    model = MBConv(x.size(1), x.size(1), 3, stride=1, p=1)
    model.train()
    output = model(x)
    x = (output == x)
    print('output size:', output.size(), 'Stochastic depth:', x[1,0,0,0])
    
class SepConv(nn.Module):
    expand = 1
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, se_scale=4, p=0.5):
        super().__init__()
        # first SepConv is not using stochastic depth
        self.p = torch.tensor(p).float() if (in_channels == out_channels) else torch.tensor(1).float()

        self.residual = nn.Sequential(
            nn.Conv2d(in_channels * SepConv.expand, in_channels * SepConv.expand, kernel_size=kernel_size,
                      stride=1, padding=kernel_size//2, bias=False, groups=in_channels*SepConv.expand),
            nn.BatchNorm2d(in_channels * SepConv.expand, momentum=0.99, eps=1e-3),
            Swish()
        )

        self.se = SEBlock(in_channels * SepConv.expand, se_scale)

        self.project = nn.Sequential(
            nn.Conv2d(in_channels*SepConv.expand, out_channels, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(out_channels, momentum=0.99, eps=1e-3)
        )

        self.shortcut = (stride == 1) and (in_channels == out_channels)

    def forward(self, x):
        # stochastic depth
        if self.training:
            if not torch.bernoulli(self.p):
                return x

        x_shortcut = x
        x_residual = self.residual(x)
        x_se = self.se(x_residual)

        x = x_se * x_residual
        x = self.project(x)

        if self.shortcut:
            x= x_shortcut + x

        return x

# check
if __name__ == '__main__':
    x = torch.randn(4, 16, 24, 24)
    model = SepConv(x.size(1), x.size(1), 3, stride=1, p=1)
    model.train()
    output = model(x)
    # stochastic depth check
    x = (output == x)
    print('output size:', output.size(), 'Stochastic depth:', x[1,0,0,0])
    
class EfficientNet(nn.Module):
    def __init__(self, num_classes=3, width_coef=1., depth_coef=1., scale=1., dropout=0.2, se_scale=4, stochastic_depth=False, p=0.5):
        super().__init__()
        channels = [32, 16, 24, 40, 80, 112, 192, 320, 1280]
        repeats = [1, 2, 2, 3, 3, 4, 1]
        strides = [1, 2, 2, 2, 1, 2, 1]
        kernel_size = [3, 3, 5, 3, 5, 5, 3]
        depth = depth_coef
        width = width_coef

        channels = [int(x*width) for x in channels]
        repeats = [int(x*depth) for x in repeats]

        # stochastic depth
        if stochastic_depth:
            self.p = p
            self.step = (1 - 0.5) / (sum(repeats) - 1)
        else:
            self.p = 1
            self.step = 0


        # efficient net
        self.upsample = nn.Upsample(scale_factor=scale, mode='bilinear', align_corners=False)

        self.stage1 = nn.Sequential(
            nn.Conv2d(4, channels[0],3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(channels[0], momentum=0.99, eps=1e-3)
        )

        self.stage2 = self._make_Block(SepConv, repeats[0], channels[0], channels[1], kernel_size[0], strides[0], se_scale)

        self.stage3 = self._make_Block(MBConv, repeats[1], channels[1], channels[2], kernel_size[1], strides[1], se_scale)

        self.stage4 = self._make_Block(MBConv, repeats[2], channels[2], channels[3], kernel_size[2], strides[2], se_scale)

        self.stage5 = self._make_Block(MBConv, repeats[3], channels[3], channels[4], kernel_size[3], strides[3], se_scale)

        self.stage6 = self._make_Block(MBConv, repeats[4], channels[4], channels[5], kernel_size[4], strides[4], se_scale)

        self.stage7 = self._make_Block(MBConv, repeats[5], channels[5], channels[6], kernel_size[5], strides[5], se_scale)

        self.stage8 = self._make_Block(MBConv, repeats[6], channels[6], channels[7], kernel_size[6], strides[6], se_scale)

        self.stage9 = nn.Sequential(
            nn.Conv2d(channels[7], channels[8], 1, stride=1, bias=False),
            nn.BatchNorm2d(channels[8], momentum=0.99, eps=1e-3),
            Swish()
        ) 

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.dropout = nn.Dropout(p=dropout)
        self.linear = nn.Linear(channels[8], num_classes)

    def forward(self, x):
        x = self.upsample(x)
        x = self.stage1(x)
        x = self.stage2(x)
        x = self.stage3(x)
        x = self.stage4(x)
        x = self.stage5(x)
        x = self.stage6(x)
        x = self.stage7(x)
        x = self.stage8(x)
        x = self.stage9(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = self.linear(x)
        return x


    def _make_Block(self, block, repeats, in_channels, out_channels, kernel_size, stride, se_scale):
        strides = [stride] + [1] * (repeats - 1)
        layers = []
        for stride in strides:
            layers.append(block(in_channels, out_channels, kernel_size, stride, se_scale, self.p))
            in_channels = out_channels
            self.p -= self.step

        return nn.Sequential(*layers)


def efficientnet_b0(num_classes=3):
    return EfficientNet(num_classes=num_classes, width_coef=1.0, depth_coef=1.0, scale=1.0,dropout=0.2, se_scale=4)

def efficientnet_b1(num_classes=3):
    return EfficientNet(num_classes=num_classes, width_coef=1.0, depth_coef=1.1, scale=240/224, dropout=0.2, se_scale=4)

def efficientnet_b2(num_classes=3):
    return EfficientNet(num_classes=num_classes, width_coef=1.1, depth_coef=1.2, scale=260/224., dropout=0.3, se_scale=4)

def efficientnet_b3(num_classes=3):
    return EfficientNet(num_classes=num_classes, width_coef=1.2, depth_coef=1.4, scale=300/224, dropout=0.3, se_scale=4)

def efficientnet_b4(num_classes=3):
    return EfficientNet(num_classes=num_classes, width_coef=1.4, depth_coef=1.8, scale=380/224, dropout=0.4, se_scale=4)

def efficientnet_b5(num_classes=3):
    return EfficientNet(num_classes=num_classes, width_coef=1.6, depth_coef=2.2, scale=456/224, dropout=0.4, se_scale=4)

def efficientnet_b6(num_classes=3):
    return EfficientNet(num_classes=num_classes, width_coef=1.8, depth_coef=2.6, scale=528/224, dropout=0.5, se_scale=4)

def efficientnet_b7(num_classes=3):
    return EfficientNet(num_classes=num_classes, width_coef=2.0, depth_coef=3.1, scale=600/224, dropout=0.5, se_scale=4)


# check
if __name__ == '__main__':
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    x = torch.randn(4, 4, 224, 224).to(device)
    model = efficientnet_b4().to(device)
    output = model(x)
    print('output size:', output.size())

output size: torch.Size([4, 4, 224, 224])
output size: torch.Size([4, 56, 1, 1])
output size: torch.Size([4, 16, 24, 24]) Stochastic depth: tensor(False)
output size: torch.Size([4, 16, 24, 24]) Stochastic depth: tensor(False)


/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:3680: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  "The default behavior for interpolate/upsample with float scale_factor changed "


output size: torch.Size([4, 3])


In [7]:
import gc
# define loss function, optimizer, lr_scheduler
loss_func = nn.CrossEntropyLoss(reduction='sum')
opt = optim.Adam(model.parameters(), lr=0.01)

from torch.optim.lr_scheduler import ReduceLROnPlateau
lr_scheduler = ReduceLROnPlateau(opt, mode='min', factor=0.1, patience=10)


# get current lr
def get_lr(opt):
    for param_group in opt.param_groups:
        return param_group['lr']


# calculate the metric per mini-batch
def metric_batch(output, target):
    pred = output.argmax(1, keepdim=True)
    corrects = pred.eq(target.view_as(pred)).sum().item()
    return corrects


# calculate the loss per mini-batch
def loss_batch(loss_func, output, target, opt=None):
    loss_b = loss_func(output, target)
    metric_b = metric_batch(output, target)

    if opt is not None:
        opt.zero_grad()
        loss_b.backward()
        opt.step()
    
    return loss_b.item(), metric_b


# calculate the loss per epochs
def loss_epoch(model, loss_func, dataset_dl, sanity_check=False, opt=None):
    running_loss = 0.0
    running_metric = 0.0
    len_data = len(dataset_dl.dataset)

    for xb, yb in dataset_dl:
        xb = xb.to(device)
        yb = yb.to(device)
        output = model(xb)

        loss_b, metric_b = loss_batch(loss_func, output, yb, opt)

        running_loss += loss_b
        
        if metric_b is not None:
            running_metric += metric_b

        if sanity_check is True:
            break

    loss = running_loss / len_data
    metric = running_metric / len_data
    return loss, metric


# function to start training
def train_val(model, params):
    num_epochs=params['num_epochs']
    loss_func=params['loss_func']
    opt=params['optimizer']
    train_dl=params['train_dl']
    val_dl=params['val_dl']
    sanity_check=params['sanity_check']
    lr_scheduler=params['lr_scheduler']
    path2weights=params['path2weights']

    loss_history = {'train': [], 'val': []}
    metric_history = {'train': [], 'val': []}

    best_loss = float('inf')
    best_model_wts = copy.deepcopy(model.state_dict())
    start_time = time.time()

    for epoch in range(num_epochs):
        current_lr = get_lr(opt)
        print('Epoch {}/{}, current lr= {}'.format(epoch, num_epochs-1, current_lr))

        model.train()
        train_loss, train_metric = loss_epoch(model, loss_func, train_dl, sanity_check, opt)
        loss_history['train'].append(train_loss)
        metric_history['train'].append(train_metric)

        model.eval()
        with torch.no_grad():
            val_loss, val_metric = loss_epoch(model, loss_func, val_dl, sanity_check)
        loss_history['val'].append(val_loss)
        metric_history['val'].append(val_metric)

        if val_loss < best_loss:
            best_loss = val_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(model.state_dict(), path2weights)
            print('Copied best model weights!')

        lr_scheduler.step(val_loss)
        if current_lr != get_lr(opt):
            print('Loading best model weights!')
            model.load_state_dict(best_model_wts)

        print('train loss: %.6f, val loss: %.6f, accuracy: %.2f, time: %.4f min' %(train_loss, val_loss, 100*val_metric, (time.time()-start_time)/60))
        print('-'*10)

    model.load_state_dict(best_model_wts)
    return model, loss_history, metric_history

In [8]:
# define the training parameters
params_train = {
    'num_epochs':60,
    'optimizer':opt,
    'loss_func':loss_func,
    'train_dl':train_dl,
    'val_dl':val_dl,
    'sanity_check':False,
    'lr_scheduler':lr_scheduler,
    'path2weights':'./models/weights.pt',
}

# check the directory to save weights.pt
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSerror:
        print('Error')
createFolder('./models')

In [9]:
model, loss_hist, metric_hist = train_val(model, params_train)

Epoch 0/59, current lr= 0.01
Copied best model weights!
train loss: 1.068896, val loss: 1.202285, accuracy: 42.73, time: 2.6703 min
----------
Epoch 1/59, current lr= 0.01
Copied best model weights!
train loss: 1.003090, val loss: 0.952421, accuracy: 56.33, time: 4.8100 min
----------
Epoch 2/59, current lr= 0.01
train loss: 0.977454, val loss: 0.958217, accuracy: 53.12, time: 7.0869 min
----------
Epoch 3/59, current lr= 0.01
Copied best model weights!
train loss: 0.968862, val loss: 0.951223, accuracy: 55.55, time: 9.2494 min
----------
Epoch 4/59, current lr= 0.01
train loss: 0.960906, val loss: 0.967320, accuracy: 50.47, time: 11.3508 min
----------
Epoch 5/59, current lr= 0.01
train loss: 0.963671, val loss: 0.966151, accuracy: 54.69, time: 13.6787 min
----------
Epoch 6/59, current lr= 0.01
Copied best model weights!
train loss: 0.957144, val loss: 0.921427, accuracy: 56.41, time: 16.0082 min
----------
Epoch 7/59, current lr= 0.01
train loss: 0.949170, val loss: 0.930315, accura

In [11]:
# No Label Data
path_dir = './PPTD'
file_list = os.listdir(path_dir)

real_data_test = [] # Tensor 로 변환된 No Label Data
for data in file_list:
    temp = np.load('./PPTD/' + data)
    tempData = torch.Tensor(temp)
    real_data_test.append(tempData)
    

data_loader = DataLoader(dataset = real_data_test, batch_size=40, shuffle = False, drop_last = True)
network = model

with torch.no_grad(): # Model 의 Back Propagation 을 막음
    for img in data_loader:
        img = img.to(device)
        pred = network(img)
        tmp = torch.argmax(pred, 1)
       
        cntAD = 0 # 1명분 40개의 발화 데이터 Output 중 AD의 개수
        cntNM = 0 # 1명분 40개의 발화 데이터 Output 중 NM의 개수
        cntPD = 0 # 1명분 40개의 발화 데이터 Output 중 PD의 개수

        for i in range(0, 40):
            if(tmp[i] == 0): # AD 일 떄 
                cntAD = cntAD + 1
            elif(tmp[i] == 1): # NM 일 때
                cntNM = cntNM + 1
            elif(tmp[i] == 2): # PD 일 떄 
                cntPD = cntPD + 1
        
        print(cntAD, cntNM, cntPD)

12 16 12
16 18 6
11 22 7
12 16 12
16 16 8
18 18 4
11 23 6
10 17 13
16 18 6
14 20 6
13 13 14
19 12 9
17 16 7
17 12 11
16 19 5
12 18 10
18 17 5
13 19 8
13 21 6
7 26 7
17 17 6
10 18 12
8 27 5
17 14 9
13 19 8
14 18 8
13 14 13
17 13 10
14 14 12
14 18 8
17 15 8
11 16 13
9 23 8
12 18 10
13 17 10
13 18 9
9 25 6
18 13 9
8 23 9
14 19 7


In [22]:
network = model

# val_ds train Data를 8 : 2 로 나눈 데이터중 validation Dataset
data_loader = DataLoader(dataset = val_ds, batch_size=40, shuffle = False, drop_last = True)

# Label 된 Data를 통해 No Label Data의 기준을 잡음
with torch.no_grad():
    for img, label in data_loader:
        img = img.to(device)
        pred = network(img)
        tmp = torch.argmax(pred, 1)
       
        cntAD = 0
        cntNM = 0
        cntPD = 0

        for i in range(0, 40):
            if(tmp[i] == 0): # AD 일 떄 
                cntAD = cntAD + 1
            elif(tmp[i] == 1): # NM 일 때
                cntNM = cntNM + 1
            elif(tmp[i] == 2): # PD 일 떄 
                cntPD = cntPD + 1
        
        print(cntAD, cntNM, cntPD)

17 16 7
13 17 10
7 24 9
7 19 14
8 21 11
8 21 11
7 22 11
9 22 9
9 23 8
10 22 8
12 20 8
12 18 10
13 19 8
7 20 13
7 23 10
11 22 7
12 24 4
13 19 8
11 21 8
14 15 11
7 25 8
12 21 7
8 21 11
11 19 10
12 21 7
8 19 13
7 24 9
15 22 3
16 15 9
8 22 10
9 22 9
5 25 10
